In [1]:
# The .json file was converted in the terminal of the local machine command used: cas anndata2cas --anndata /Users/aa37/Documents/GitHub/siletti_hacking/Anndata_source/Siletti_all_neurons.h5ad --labelsets subcluster_id --output /Users/aa37/Documents/GitHub/siletti_hacking/intermediatte_files/Siletti_subclusters_from_anndata_cas_all_neurons.json

In [2]:
import json

#  Open files

with open('./intermediatte_files/Siletti_from_abc.json', 'r') as f:
    Siletti_abc_cas = json.loads(f.read())

with open('./cas_source/CS202210140_non_neuronal.json', 'r') as f:
    nn = json.loads(f.read())
with open('./cas_source/CS202210140_neurons.json', 'r') as f:
    neurons = json.loads(f.read())

#Generate lookups for accessions for nn and neuron clusters respectively
neuron_clusters = { a['cell_set_accession'] for a in neurons['annotations'] 
                    if a['labelset'] == 'Cluster'}
nn_clusters = { a['cell_set_accession'] for a in nn['annotations'] 
                    if a['labelset'] == 'Cluster'}

# Make lists of subclusters from ABC that have a parent cluster in nn & neurons respectively
nn_subclusters = [a for a in Siletti_abc_cas['annotations']
                  if (a['labelset'] == 'subcluster') 
                  and (a['parent_cell_set_accession'] in nn_clusters)]
neuron_subclusters = [a for a in Siletti_abc_cas['annotations']
                  if (a['labelset'] == 'subcluster') 
                  and (a['parent_cell_set_accession'] in neuron_clusters)]

# Merge in subclusters

nn['annotations'].extend(nn_subclusters)
neurons['annotations'].extend(neuron_subclusters)

In [3]:
# Generate lookup for subcluster cell_ids, keyed on subclusters_id

# Open files
import json
with open('./intermediatte_files/Siletti_subclusters_from_anndata_cas.json', 'r') as f:
    Siletti_subclusters_from_ad = json.loads(f.read())

subcluster_cell_id_lookup = { a['cell_label'] : a['cell_ids'] for a in Siletti_subclusters_from_ad['annotations']}

In [4]:
# Check number of matches on both sides
len(subcluster_cell_id_lookup.keys())

297

In [5]:
len(Siletti_subclusters_from_ad['annotations'])

297

In [6]:
len(nn_subclusters)

297

In [7]:
nn_subclusters[0]

{'labelset': 'subcluster',
 'cell_label': 'Oligo_50_2996',
 'cell_set_accession': 'CS202210140_3490',
 'parent_cell_set_accession': 'CS202210140_51'}

In [8]:
nn_clusters

{'CS202210140_10',
 'CS202210140_11',
 'CS202210140_12',
 'CS202210140_13',
 'CS202210140_14',
 'CS202210140_15',
 'CS202210140_16',
 'CS202210140_17',
 'CS202210140_18',
 'CS202210140_19',
 'CS202210140_20',
 'CS202210140_21',
 'CS202210140_22',
 'CS202210140_23',
 'CS202210140_24',
 'CS202210140_25',
 'CS202210140_26',
 'CS202210140_27',
 'CS202210140_28',
 'CS202210140_29',
 'CS202210140_30',
 'CS202210140_31',
 'CS202210140_32',
 'CS202210140_33',
 'CS202210140_34',
 'CS202210140_35',
 'CS202210140_36',
 'CS202210140_37',
 'CS202210140_38',
 'CS202210140_39',
 'CS202210140_40',
 'CS202210140_41',
 'CS202210140_42',
 'CS202210140_43',
 'CS202210140_44',
 'CS202210140_45',
 'CS202210140_46',
 'CS202210140_47',
 'CS202210140_48',
 'CS202210140_49',
 'CS202210140_5',
 'CS202210140_50',
 'CS202210140_51',
 'CS202210140_52',
 'CS202210140_53',
 'CS202210140_54',
 'CS202210140_55',
 'CS202210140_56',
 'CS202210140_57',
 'CS202210140_58',
 'CS202210140_59',
 'CS202210140_6',
 'CS202210140_

In [9]:
import re
# Testing regex pattern
m = re.match('.+_(\d+)', 'URL_297_0')
m.group(1)

'0'

In [10]:
# Use regex matching ID -> label to attach cell_ids to subclusters
for c in nn_subclusters:
    m = re.match('.+_(\d+)', c['cell_label'])
    if int(m.group(1)) in subcluster_cell_id_lookup.keys():
        c['cell_ids'] = subcluster_cell_id_lookup[int(m.group(1))]

In [11]:
# Test addition cell_Ids has worked
nn_subclusters[0]['cell_ids'][0:5]

['10X362_3:TCAGTGAGTATTGACC',
 '10X362_5:TCCGTGTGTGAAAGTT',
 '10X362_5:CACGGGTAGAGCAGAA',
 '10X362_6:AGGACTTGTATCCTTT',
 '10X362_6:GCATTAGGTTGGACCC']

In [12]:
import pandas as pd
AT = pd.read_csv('./Annotation_transfer/Clusters_AT_from_MTG.tsv', sep='\t')
MTG_annotations = pd.read_csv('./MTG_cas/CCN20240304_annotations.tsv', sep='\t')
MTG_annotations[0:5]

,cell_set_accession,cell_label,cell_fullname,parent_cell_set_accession,parent_cell_set_name,labelset,cell_ontology_term_id,cell_ontology_term,rationale,rationale_dois,marker_gene_evidence,synonyms
0,b15b444731,Vip_2,Vip_2,082c817b6f,Vip,CrossArea_cluster,CL:4023016,vip GABAergic cortical interneuron,NaN,NaN,NaN,NaN
1,83796aa6d9,Sst_11,Sst_11,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
2,8c25f3eded,Sst_31,Sst_31,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
3,319ad7ab6a,Sst_37,Sst_37,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
4,51b6c3ddc3,L6 CT_3,L6 CT_3,d4ee4a2c4c,L6 CT,CrossArea_cluster,CL:4023013,corticothalamic-projecting glutamatergic corti...,NaN,NaN,NaN,NaN


In [13]:
# Aim 
# Look up transfered MTG names (in MT) in MTG cell_label var - MT - & link Siletti Cell label to MTG annotation object

# 1. Make template for annotation transfer objects
# 2. Iterate over annotatoin transfer labels, matching to cluster labels & using this to generate annotation transfer objects.
# 3. Attach annotation transfer objects to annotatinons

In [14]:
MTG_annotation_lookup = MTG_annotations.set_index("cell_label").to_dict(orient='index')

In [15]:
# What matches?
set(MTG_annotation_lookup.keys()).intersection(set(AT['Transferred MTG Label']))

{'Astro',
 'Chandelier',
 'Endo',
 'L2/3 IT',
 'L4 IT',
 'L5 IT',
 'L5/6 NP',
 'L6 CT',
 'L6 IT',
 'L6 IT Car3',
 'L6b',
 'Lamp5',
 'Lamp5 Lhx6',
 'OPC',
 'Oligo',
 'Pax6',
 'Pvalb',
 'Sncg',
 'Sst',
 'VLMC',
 'Vip'}

In [16]:
# What doesn't match?  - Can we match this by hand?
(set(AT['Transferred MTG Label']))-set(MTG_annotation_lookup.keys())

{'Micro-PVM', nan}

In [17]:
# Dict comp => Dict keyed on Siletti Cluster_name (=cell_label) with value = MTG annotation transfer object
# Iterate over Transferred annotations (AT)
out = {}
comment = "We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8."
for i,r in AT.iterrows(): 
    if r['Transferred MTG Label'] in MTG_annotation_lookup.keys():
        MTG_ann =  MTG_annotation_lookup[r['Transferred MTG Label']]
        out[r['Cluster name']]= { "transferred_cell_label" : r['Transferred MTG Label'], 
                                  "source_node_accession": MTG_ann['labelset'] + '_' + MTG_ann['cell_set_accession'],
                                  "source_taxonomy": "https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json",
                                  "comment": comment }
# Test
out['Amex_175']

{'transferred_cell_label': 'L2/3 IT',
 'source_node_accession': 'CrossArea_subclass_58570ade17',
 'source_taxonomy': 'https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json',
 'comment': 'We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8.'}

In [18]:
# Attach Annotation transfers to CAS for neurons matching on names

for a in nn['annotations']:
    if a['cell_label'] in out.keys():
        a['annotation_transfer'] = [out[a['cell_label']]]
        # Testing
[a['annotation_transfer'] for a in nn['annotations'] if 'annotation_transfer' in a.keys()][0]

[{'transferred_cell_label': 'Endo',
  'source_node_accession': 'CrossArea_subclass_48e48631ba',
  'source_taxonomy': 'https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json',
  'comment': 'We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8.'}]

In [19]:
import os
import json
import random
import anndata
import pandas as pd
import urllib.request as request

from cas.file_utils import read_json_file, write_dict_to_json_file
from cas.add_author_annotations import add_author_annotations

In [20]:
# Load the CSV file
csv_df = pd.read_csv('./Author_cathegories/science.add7046_table_s3.csv')
# filter empty rows
csv_df = csv_df[csv_df['cell_label'].notna()]
csv_df.head(3)

,Cluster ID,cell_label,Supercluster,Class auto-annotation,Neurotransmitter auto-annotation,Neuropeptide auto-annotation,Subtype auto-annotation,Transferred MTG Label,Top three regions,Top three dissections,...,DoubletFinder score,Total UMI,Fraction unspliced,Fraction mitochondrial,H19.30.002,H19.30.001,H18.30.002,H18.30.001,Fraction cells from top donor,Number of donors
0,0.0,Bcell_0,Miscellaneous,BCELL,NaN,NaN,NaN,NaN,"Midbrain: 21.0%, Basal forebrain: 19.0%, Pons:...","Human GPi: 11.4%, Human SN: 11.4%, Human SN-RN...",...,0.013358,2258.866667,0.608113,0.020989,58.0,20.0,27.0,0.0,0.552381,3.0
1,1.0,Tcell_1,Miscellaneous,TCELL,NaN,NaN,NaN,NaN,"Midbrain: 15.0%, Basal forebrain: 14.0%, Pons:...","Human SpC: 8.2%, Human CbDN: 7.6%, Human SN: 7.4%",...,0.022371,2184.314470,0.582884,0.021212,708.0,400.0,1364.0,2.0,0.551334,4.0
2,2.0,Nkcell_2,Miscellaneous,NK,NaN,NaN,NaN,NaN,"Thalamus: 17.6%, Cerebral cortex: 15.5%, Basal...","Human GPe: 5.7%, Human GPi: 5.5%, Human CbDN: ...",...,0.023788,2263.348733,0.558228,0.020093,245.0,126.0,300.0,0.0,0.447094,3.0


In [21]:
csv_columns = csv_df.columns.tolist()
exclude_columns = ["Cluster ID", "cell_label", "Supercluster"]
columns_to_ingest = [column for column in csv_columns if column not in exclude_columns]
columns_to_ingest

['Class auto-annotation',
 'Neurotransmitter auto-annotation',
 'Neuropeptide auto-annotation',
 'Subtype auto-annotation',
 'Transferred MTG Label',
 'Top three regions',
 'Top three dissections',
 'Top Enriched Genes',
 'Number of cells',
 'DoubletFinder score',
 'Total UMI',
 'Fraction unspliced',
 'Fraction mitochondrial',
 'H19.30.002',
 'H19.30.001',
 'H18.30.002',
 'H18.30.001',
 'Fraction cells from top donor',
 'Number of donors']

In [22]:
# cell_label is required, add it forcefully if not exist
for annotation in nn["annotations"]:
    if "cell_label" not in annotation:
        print(annotation)
        annotation["cell_label"] = "NA"

In [23]:
cellset_labels = [annotation["cell_label"] for annotation in nn["annotations"]]
csv_df = csv_df[csv_df['cell_label'].isin(cellset_labels)]
new_labelsets = [
    {
        "name": "subcluster",
        "description": "The finest level of cell type definition in the human whole brain taxonomy. It is defined by applying Louvain clustering independently for each cluster. Cells within a subcluster share similar characteristics and belong to the same cluster. In some cases subclusters are distinct, and in other cases represent cell states or gradients within the same cluster.",
        "rank": 0
    },
    {
        "name": "cluster",
        "description": "An intermediate level of cell type definitions in the human whole brain taxonomy. It is defined by applying Louvain clustering independently for each supercluster. All cells within a subcluster belong to the same cluster. Clusters group together similar subclusters and are highly distinct from one another.",
        "rank": 1
    },
    {
        "name": "supercluster",
        "description": "The top level of cell type definition in the human whole brain taxonomy. It is defined using Louvain clustering (Cytograph) with most superclusters determined by broad brain region and neurotransmitter type. All cells within a cluster belong to the same supercluster. Supercluster provides a broader characterization of cell types.",
        "rank": 2
    }
]

In [24]:
updated_nn = add_author_annotations(nn, csv_df, "cell_label", columns_to_ingest)
updated_nn["labelsets"] = new_labelsets

In [25]:
write_dict_to_json_file('./Final_json_files/Siletti_added_author_categories_v2_nn.json', updated_nn)